In [6]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import src.mnist

In [38]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [10]:
train_set, test_set = src.mnist.get_mnist_full()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
MNIST(N=55000, dataset="train", labels=(55000,), images=(55000, 784), PCA=False, KMeans=False)
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
MNIST(N=10000, dataset="test", labels=(10000,), images=(10000, 784), PCA=False, KMeans=False)


In [13]:
pca = PCA(n_components=100)
pca.fit(train_set.images)

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [28]:
train_images_pca = pca.transform(train_set.images)
test_images_pca = pca.transform(test_set.images)

What are all the various decision tree algorithms and how do they differ from each other? Which one is implemented in scikit-learn?
ID3 (Iterative Dichotomiser 3) was developed in 1986 by Ross Quinlan. The algorithm creates a multiway tree, finding for each node (i.e. in a greedy manner) the categorical feature that will yield the largest information gain for categorical targets. Trees are grown to their maximum size and then a pruning step is usually applied to improve the ability of the tree to generalise to unseen data.
C4.5 is the successor to ID3 and removed the restriction that features must be categorical by dynamically defining a discrete attribute (based on numerical variables) that partitions the continuous attribute value into a discrete set of intervals. C4.5 converts the trained trees (i.e. the output of the ID3 algorithm) into sets of if-then rules. These accuracy of each rule is then evaluated to determine the order in which they should be applied. Pruning is done by removing a rule’s precondition if the accuracy of the rule improves without it.
C5.0 is Quinlan’s latest version release under a proprietary license. It uses less memory and builds smaller rulesets than C4.5 while being more accurate.
CART (Classification and Regression Trees) is very similar to C4.5, but it differs in that it supports numerical target variables (regression) and does not compute rule sets. CART constructs binary trees using the feature and threshold that yield the largest information gain at each node.
scikit-learn uses an optimised version of the CART algorithm.

In [29]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_images_pca, train_set.labels)

In [30]:
labels_predictions = []
for i in range(0, test_images_pca.shape[0]):
    label = test_set.labels[i]
    prediction = clf.predict(test_images_pca[i].reshape(1, -1))
    labels_predictions.append([label, prediction])

In [31]:
def get_confusion_matrix(n_classes, label_predictions):
    ret = np.zeros((n_classes, n_classes))
    for label_prediction in label_predictions:
        label = label_prediction[0]
        prediction = label_prediction[1]
        ret[label, prediction] += 1
    return ret

In [32]:
def get_error_ratio(label_predictions):
    n_errors = 0
    for label_prediction in label_predictions:
        label = label_prediction[0]
        prediction = label_prediction[1]
        if label != prediction:
            n_errors += 1
    return float(n_errors) / float(len(label_predictions))

In [33]:
confusion_matrix = get_confusion_matrix(10, labels_predictions)

In [34]:
np.set_printoptions(suppress=True, threshold=10000)
print(confusion_matrix)

[[  868.     2.    15.    10.     9.    19.    31.     6.    16.     4.]
 [    0.  1097.     2.     4.     5.     4.     4.     5.    10.     4.]
 [   15.     5.   836.    35.    13.    22.    30.    22.    42.    12.]
 [   13.     8.    20.   812.     5.    62.     9.    13.    52.    16.]
 [    1.     6.    16.     4.   787.    11.    16.    31.    14.    96.]
 [   23.     5.     9.    68.    18.   672.    22.    12.    40.    23.]
 [   20.     3.    14.     3.    17.    22.   858.     6.    10.     5.]
 [    6.     9.    31.    21.    19.     4.     5.   870.     9.    54.]
 [   13.     8.    39.    40.    14.    39.    13.    15.   759.    34.]
 [   10.     3.    10.     8.    82.    31.     8.    48.    27.   782.]]


In [35]:
error_ratio = get_error_ratio(labels_predictions)

In [36]:
print(error_ratio)

0.1659


In [52]:
clf = RandomForestClassifier(n_estimators=50)
clf = clf.fit(train_images_pca, train_set.labels)

In [53]:
labels_predictions = []
for i in range(0, test_images_pca.shape[0]):
    label = test_set.labels[i]
    prediction = clf.predict(test_images_pca[i].reshape(1, -1))
    labels_predictions.append([label, prediction])

In [54]:
confusion_matrix = get_confusion_matrix(10, labels_predictions)

In [55]:
np.set_printoptions(suppress=True, threshold=10000)
print(confusion_matrix)

[[  962.     0.     3.     1.     1.     3.     6.     1.     3.     0.]
 [    0.  1118.     4.     2.     0.     1.     5.     1.     3.     1.]
 [   11.     0.   965.    13.     8.     2.     3.     8.    19.     3.]
 [    3.     0.     7.   953.     1.    12.     3.    10.    17.     4.]
 [    1.     2.     5.     2.   928.     3.     9.     2.     5.    25.]
 [    5.     1.     5.    25.     6.   827.    11.     1.     5.     6.]
 [    6.     3.     2.     2.     4.     7.   933.     0.     1.     0.]
 [    0.     7.    20.     2.     9.     1.     0.   963.     3.    23.]
 [    8.     0.    17.    24.     7.    18.     5.     7.   879.     9.]
 [    5.     5.     5.    13.    26.     6.     1.    12.     8.   928.]]


In [56]:
error_ratio = get_error_ratio(labels_predictions)

In [57]:
print(error_ratio)

0.0544
